# Collabrative Filtering (Item Based)
Book Recomendation using Goodreads data

In [0]:
#Import necessary Libraries


import numpy as np
import pandas as pd
import os


In [67]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:

# Load necessary CSV Files - ratings.csv, books.csv.


ratingsDF = pd.read_csv(os.path.join("drive","My Drive","Colab Notebooks", "goodbooks-10k","ratings.csv"))
bookDF = pd.read_csv(os.path.join("drive","My Drive","Colab Notebooks", "goodbooks-10k","books.csv"))
bookDF=bookDF.drop(['image_url','small_image_url','title','best_book_id','isbn','isbn13'],axis=1)

In [69]:
#contents in the books.csv file
bookDF.head(5)


,id,book_id,work_id,books_count,authors,original_publication_year,original_title,language_code,average_rating,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5
0,1,2767052,2792775,272,Suzanne Collins,2008.0,The Hunger Games,eng,4.34,4780653,4942365,155254,66715,127936,560092,1481305,2706317
1,2,3,4640799,491,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,eng,4.44,4602479,4800065,75867,75504,101676,455024,1156318,3011543
2,3,41865,3212258,226,Stephenie Meyer,2005.0,Twilight,en-US,3.57,3866839,3916824,95009,456191,436802,793319,875073,1355439
3,4,2657,3275794,487,Harper Lee,1960.0,To Kill a Mockingbird,eng,4.25,3198671,3340896,72586,60427,117415,446835,1001952,1714267
4,5,4671,245494,1356,F. Scott Fitzgerald,1925.0,The Great Gatsby,eng,3.89,2683664,2773745,51992,86236,197621,606158,936012,947718


In [70]:
#contents in the ratings.csv file
ratingsDF.head(5)

,book_id,user_id,rating
0,1,314,5
1,1,439,3
2,1,588,5
3,1,1169,4
4,1,1185,4


In [79]:
ratingsDF.isnull().values.any() #Checking for null values

False

In [81]:
len(ratingsDF) #Total length of ratings table 

981756

In [83]:
print(len(ratingsDF.user_id.unique())) #uniques users
print(len(ratingsDF.book_id.unique())) #unique books

53424
10000


In [0]:
#Buiding a list of dictionaries where each dictionary represents a book. user_id is the key and corresponding rating is its value. 

listOfDictonaries=[]
indexMap = {}
reverseIndexMap = {}
ptr=0;
testdf = ratingsDF
testdf=testdf[['user_id','rating']].groupby(testdf['book_id']) # group the ratings table by book id

#testdf.groups -------> To view the groups created
#testdf.groups.keys() --------------> Gives the keys i.e book ids




for groupKey in testdf.groups.keys():
    tempDict={}

    groupDF = testdf.get_group(groupKey)   #get a group(consists od rating and userid) of particular book_id

    for i in range(0,len(groupDF)):
        tempDict[groupDF.iloc[i,0]]=groupDF.iloc[i,1]
    indexMap[ptr]=groupKey                #A dictionary where key is indexed value(0,1,2 ....) and value is book_id
    reverseIndexMap[groupKey] = ptr       #Reverse of indexMap. Here book_id is key and index is value
    ptr=ptr+1
    listOfDictonaries.append(tempDict)



**COSINE SIMILARITY: ** is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them.

The cosine of two non-zero vectors can be derived by using the Euclidean dot product formula:

A.B = |A||B|cos$\Theta$

cos$\Theta$ = A.B/(|A|.|B|)


In [0]:
#We then use sklearn's DictVectorizer() function to create vectors corresponding to each book. We are trying to create a vector space with users as column vectors. Each point in the vector space represents a book. Rating of the book given an user is its magnitude. We then calculate similarity/distance between books in this vector space.



from sklearn.feature_extraction import DictVectorizer
dictVectorizer = DictVectorizer(sparse=True)
vector = dictVectorizer.fit_transform(listOfDictonaries)
#print(vector)

In [73]:
#We finally use sklearn's consine_similarity function to calculate pairwise similarity matrix
from sklearn.metrics.pairwise import cosine_similarity
pairwiseSimilarity = cosine_similarity(vector)
print(pairwiseSimilarity) #gives a matrix of distances between 2 vectors

[[1.         0.55552614 0.50915226 ... 0.         0.         0.        ]
 [0.55552614 1.         0.52021603 ... 0.         0.         0.00692082]
 [0.50915226 0.52021603 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.         0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.00692082 0.         ... 0.         0.         1.        ]]


**numpy.argsort(): ** It is a function is used to perform an indirect sort along the given axis using the algorithm specified by the kind keyword. It returns an array of indices of the same shape as arr that that would sort the array.

In [0]:
#At last we define a function that takes an input id and outputs 5 top most recomendations

def printBookDetails(bookID):
    print("Title:", bookDF[bookDF['id']==bookID]['original_title'].values[0])
    print("Author:",bookDF[bookDF['id']==bookID]['authors'].values[0])
    print("Printing Book-ID:",bookID)
    print("=================++++++++++++++=========================")


def getTopRecommandations(bookID):
    row = reverseIndexMap[bookID]
    print("------INPUT BOOK--------")
    printBookDetails(bookID)
    print("-------RECOMMENDATIONS----------")
    
    #np.argsort(pairwiseSimilarity[row]) gives the sorted array of similarities of given row. This sorts in ascendending order of similarity
    #But we need highest simialr matrix. So we fetch from last.But the last first book is the book itself. So we leave that fetch from second last one using [-7:-2] i.e it fetches from last 7th element to last 2nd element. TO reverse that order we use [::-1] 
    
    similarBookIDs = [printBookDetails(indexMap[i]) for i in np.argsort(pairwiseSimilarity[row])[-7:-2][::-1]] 


In [76]:
# id is the column id in book.csv. Values of id are in [1,10000]
getTopRecommandations(123)


------INPUT BOOK--------
Title: The Firm
Author: John Grisham
Printing Book-ID: 123
=================++++++++++++++=========================
-------RECOMMENDATIONS----------
Title: The Pelican Brief
Author: John Grisham
Printing Book-ID: 281
=================++++++++++++++=========================
Title: Angels & Demons 
Author: Dan Brown
Printing Book-ID: 9
=================++++++++++++++=========================
Title: The Client
Author: John Grisham
Printing Book-ID: 227
=================++++++++++++++=========================
Title: Divine Secrets of the Ya-Ya Sisterhood
Author: Rebecca Wells
Printing Book-ID: 136
=================++++++++++++++=========================
Title: The Secret Life of Bees
Author: Sue Monk Kidd
Printing Book-ID: 57
=================++++++++++++++=========================
